# Posted Workers in the EU, a fact sheet

*Version 1. October 23rd, 2017*<br>
&#9733;<i>[Thomas Roca](https://twitter.com/Thomas_Roca), PhD - Economist & Data Strategist @ Microsoft Europe &#9733;</i>

---

<img src="https://raw.githubusercontent.com/ThomasRoca/Posted-Workers/master/pic2.png" width='75%'>

---

October the 23rd the Employment, Social Policy, Health and Consumer Affairs Council will disscussed the European directived on posted workers. 


### But what is a posted worker ?
As stated by the European Commission, 'a "posted worker" is an employee who is sent by his employer to carry out a service in another EU Member State on a temporary basis.' [More here: http://on ec.europea.eu](http://ec.europa.eu/social/main.jsp?catId=471)

These temporary workers have recently garnered significant attention, especially in France and Poland. Here are some examples from news outlets: [Euronews](http://www.euronews.com/2017/09/29/posted-workers-a-french-paradox), or [euractiv](https://www.euractiv.com/section/economy-jobs/news/france-and-poland-hopeful-about-posted-workers-breakthrough/), or [the news.pl](http://www.thenews.pl/1/10/Artykul/331422,More-debate-needed-on-migration-posted-workers-Polish-PM-after-EU-summit)

### But how massive is this movement? How mobile are temporary workers in the EU? Let's crunch numbers...
The EU commission publishes annual reports on the topic. Data from these reports are extracted from A1 forms and published as tables. For this fact sheet we used the [latest report](http://ec.europa.eu/social/BlobServlet?docId=17164&langId=en) that is available on the Commission web page: "Posting of workers - Report on A1 portable documents issued in 2015".


The bad news is that the data is not available (yet) on the [EU open data portal](https://data.europa.eu/euodp/en/home/). Thus, we have to extract it from the PDF which can lead to errors.The CSV for 2015 is available on this [Posted Workers Github rep](https://github.com/ThomasRoca/Posted-Workers)
NB. Thanks to **Mathias Busse** from **[CEPS](https://www.ceps.eu/)** for sharing the file.

### Making Sense of the data

To understand the flows of 'posted workers', we need to visualize it. In this notebook I will explain how to prepare the data and show two types of visualization:
- A flow chart using [Highcharts](https://www.highcharts.com/)
- A Dashboard using [Microsoft PowerBI](https://powerbi.microsoft.com)

In order to do so, some work is necessary to shape the dataset. I wrote a small Python script to transform the Matrix-shaped  database into one-to-one flows.

### Preparing the data

#### First, load the dataset:

In [30]:
import pandas as pd
#URL of the file
path="https://raw.githubusercontent.com/ThomasRoca/Posted-Workers/master/"
#Name of the file
filename="Posted%20workers%2C%20sending%20countries.csv"
#Read using pandas
dataset = pd.read_csv(path+filename, thousands=',')
#Re-index the file
col_list = dataset.cname.values
dataset = dataset.set_index('cname')
dataset.tail(10)

,BE,BG,CZ,DK,DE,EE,IE,EL,ES,FR,...,SK,FI,SE,UK,Unnamed: 29,IS,LI,NO,CH,Total
cname,,,,,,,,,,,,,,,,,,,,,
SK,104,76,988.0,11.0,1951.0,0.0,3.0,2.0,339.0,635.0,...,8.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,8141.0
FI,111,151,345.0,54.0,3019.0,3282.0,178.0,47.0,566.0,728.0,...,868.0,5.0,177.0,NaN,NaN,NaN,NaN,NaN,NaN,18618.0
SE,384,152,254.0,718.0,8143.0,324.0,701.0,7.0,1075.0,1662.0,...,2245.0,947.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,37373.0
UK,2341,530,262.0,332.0,11069.0,8.0,690.0,226.0,9886.0,11864.0,...,1127.0,218.0,162.0,NaN,NaN,20.0,NaN,NaN,NaN,54344.0
IS,12,0,19.0,4.0,67.0,0.0,8.0,NaN,102.0,88.0,...,5.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,608.0
LI,7,0,3.0,276.0,0.0,0.0,3.0,NaN,28.0,0.0,...,7.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,803.0
NO,281,202,122.0,3889.0,3552.0,749.0,119.0,7.0,713.0,1111.0,...,721.0,464.0,1639.0,NaN,NaN,42.0,NaN,NaN,NaN,24958.0
CH,1282,98,359.0,47.0,33627.0,2.0,37.0,65.0,2077.0,9014.0,...,1117.0,71.0,48.0,NaN,NaN,4.0,NaN,NaN,NaN,97701.0
unkn.,96,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we can see, the dataset need to be cleaned. We want to get rid of "unkn." and its variations but also the 'total' row.
Ultimately, for the visualization I have in mind, I need to transform this data into a JSON for each flow, something like:

```javascript
[{"origin":"Belgium","destination":"Bulgaria","value": 42 },
{"origin":"Belgium","destination":"Czech Republic","value": 399 },
//...
]

```

I prefer using actual country names rather than the ISO 2 letters code, that we are not all familiar with (especially since EU sometime use its own, like HE for Greece and not GR...). To 'convert' the iso 2 letters code into Country name I rely on dictionnaries.

### Creating the JSON

In [29]:
import numpy
import codecs

#Here I could have used a dictionnary that contains iso2 codes and country names...
#... but I didn't had this one handy so i used those ones and nested it like this: iso3_country_dict[iso2_iso3_dict[iso2]]
iso3_country_dict = {"STP":"Sao Tome and Principe","ASM":"American Samoa","AUS":"Australia","BRN":"Brunei Darussalam","CHN":"China","FJI":"Fiji","FSM":"Micronesia, Fed. Sts.","GUM":"Guam","HKG":"Hong Kong SAR, China","IDN":"Indonesia","JPN":"Japan","KHM":"Cambodia","KIR":"Kiribati","KOR":"Korea, Rep.","LAO":"Lao PDR","MAC":"Macao SAR, China","MHL":"Marshall Islands","MMR":"Myanmar","MNG":"Mongolia","MNP":"Northern Mariana Islands","MYS":"Malaysia","NCL":"New Caledonia","NZL":"New Zealand","PHL":"Philippines","PLW":"Palau","PNG":"Papua New Guinea","PRK":"Korea, Dem. Rep.","PYF":"French Polynesia","SGP":"Singapore","SLB":"Solomon Islands","THA":"Thailand","TLS":"Timor-Leste","TON":"Tonga","TUV":"Tuvalu","TWN":"Taiwan, China","VNM":"Vietnam","VUT":"Vanuatu","WSM":"Samoa","ALB":"Albania","AND":"Andorra","ARM":"Armenia","AUT":"Austria","AZE":"Azerbaijan","BEL":"Belgium","BGR":"Bulgaria","BIH":"Bosnia and Herzegovina","BLR":"Belarus","CHE":"Switzerland","CHI":"Channel Islands","CYP":"Cyprus","CZE":"Czech Republic","DEU":"Germany","DNK":"Denmark","ESP":"Spain","EST":"Estonia","FIN":"Finland","FRA":"France","FRO":"Faeroe Islands","GBR":"United Kingdom","GEO":"Georgia","GRC":"Greece","GRL":"Greenland","HRV":"Croatia","HUN":"Hungary","IMN":"Isle of Man","IRL":"Ireland","ISL":"Iceland","ITA":"Italy","KAZ":"Kazakhstan","KGZ":"Kyrgyz Republic","LIE":"Liechtenstein","LTU":"Lithuania","LUX":"Luxembourg","LVA":"Latvia","MCO":"Monaco","MDA":"Moldova","MKD":"Macedonia, FYR","MNE":"Montenegro","NLD":"Netherlands","NOR":"Norway","POL":"Poland","PRT":"Portugal","ROU":"Romania","RUS":"Russian Federation","SMR":"San Marino","SRB":"Serbia","SVK":"Slovakia","SVN":"Slovenia","SWE":"Sweden","TJK":"Tajikistan","TKM":"Turkmenistan","TUR":"Turkey","UKR":"Ukraine","UZB":"Uzbekistan","ABW":"Aruba","ARG":"Argentina","ATG":"Antigua and Barbuda","BHS":"Bahamas, The","BLZ":"Belize","BOL":"Bolivia","BRA":"Brazil","BRB":"Barbados","CHL":"Chile","COL":"Colombia","CRI":"Costa Rica","CUB":"Cuba","CUW":"Curacao","CYM":"Cayman Islands","DMA":"Dominica","DOM":"Dominican Republic","ECU":"Ecuador","GRD":"Grenada","GTM":"Guatemala","GUY":"Guyana","HND":"Honduras","HTI":"Haiti","JAM":"Jamaica","KNA":"St. Kitts and Nevis","LCA":"St. Lucia","MAF":"St. Martin (French part)","MEX":"Mexico","NIC":"Nicaragua","PAN":"Panama","PER":"Peru","PRI":"Puerto Rico","PRY":"Paraguay","SLV":"El Salvador","SUR":"Suriname","SXM":"Sint Maarten (Dutch part)","TCA":"Turks and Caicos Islands","TTO":"Trinidad and Tobago","URY":"Uruguay","VCT":"St. Vincent and the Grenadines","VEN":"Venezuela, RB","VIR":"Virgin Islands (U.S.)","ARE":"United Arab Emirates","BHR":"Bahrain","DJI":"Djibouti","DZA":"Algeria","EGY":"Egypt, Arab Rep.","IRN":"Iran, Islamic Rep.","IRQ":"Iraq","ISR":"Israel","JOR":"Jordan","KWT":"Kuwait","LBN":"Lebanon","LBY":"Libya","MAR":"Morocco","MLT":"Malta","OMN":"Oman","PSE":"West Bank and Gaza","QAT":"Qatar","SAU":"Saudi Arabia","SYR":"Syrian Arab Republic","TUN":"Tunisia","YEM":"Yemen, Rep.","BMU":"Bermuda","CAN":"Canada","USA":"United States","AFG":"Afghanistan","BGD":"Bangladesh","BTN":"Bhutan","IND":"India","LKA":"Sri Lanka","MDV":"Maldives","NPL":"Nepal","PAK":"Pakistan","AGO":"Angola","BDI":"Burundi","BEN":"Benin","BFA":"Burkina Faso","BWA":"Botswana","CAF":"Central African Republic","CIV":"Cote d'Ivoire","CMR":"Cameroon","COD":"Congo, Dem. Rep.","COG":"Congo, Rep.","COM":"Comoros","CPV":"Cabo Verde","ERI":"Eritrea","ETH":"Ethiopia","GAB":"Gabon","GHA":"Ghana","GIN":"Guinea","GMB":"Gambia, The","GNB":"Guinea-Bissau","GNQ":"Equatorial Guinea","KEN":"Kenya","LBR":"Liberia","LSO":"Lesotho","MDG":"Madagascar","MLI":"Mali","MOZ":"Mozambique","MRT":"Mauritania","MUS":"Mauritius","MWI":"Malawi","NAM":"Namibia","NER":"Niger","NGA":"Nigeria","RWA":"Rwanda","SDN":"Sudan","SEN":"Senegal","SLE":"Sierra Leone","SOM":"Somalia","SSD":"South Sudan","SWZ":"Swaziland","SYC":"Seychelles","TCD":"Chad","TGO":"Togo","TZA":"Tanzania","UGA":"Uganda","ZAF":"South Africa","ZMB":"Zambia","ZWE":"Zimbabwe"} 
iso2_iso3_dict ={"CZ":"CZE","LI":"LIE","UK":"GBR", "EL":"GRC" ,"AF":"AFG","AL":"ALB","DZ":"DZA","AO":"AGO","AG":"ATG","AR":"ARG","AM":"ARM","AU":"AUS","AT":"AUT","AZ":"AZE","BS":"BHS","BH":"BHR","BD":"BGD","BB":"BRB","BY":"BLR","BE":"BEL","BZ":"BLZ","BJ":"BEN","BT":"BTN","BO":"BOL","BA":"BIH","BW":"BWA","BR":"BRA","BN":"BRN","BG":"BGR","BF":"BFA","BI":"BDI","CV":"CPV","KH":"KHM","CM":"CMR","CA":"CAN","CF":"CAF","TD":"TCD","CL":"CHL","CN":"CHN","CO":"COL","KM":"COM","CD":"COD","CG":"COG","CR":"CRI","CI":"CIV","HR":"HRV","CY":"CYP","CZ":"CZE","DK":"DNK","DJ":"DJI","DM":"DMA","DO":"DOM","EC":"ECU","EG":"EGY","SV":"SLV","GQ":"GNQ","ER":"ERI","EE":"EST","ET":"ETH","FJ":"FJI","FI":"FIN","FR":"FRA","GA":"GAB","GM":"GMB","GE":"GEO","DE":"DEU","GH":"GHA","GR":"GRC","GD":"GRD","GT":"GTM","GN":"GIN","GW":"GNB","GY":"GUY","HT":"HTI","HN":"HND","HK":"HKG","HU":"HUN","IS":"ISL","IN":"IND","ID":"IDN","IR":"IRN","IQ":"IRQ","IE":"IRL","IL":"ISR","IT":"ITA","JM":"JAM","JP":"JPN","JO":"JOR","KZ":"KAZ","KE":"KEN","KI":"KIR","KR":"KOR","XK":"UNK","KW":"KWT","KG":"KGZ","LA":"LAO","LV":"LVA","LB":"LBN","LS":"LSO","LR":"LBR","LY":"LBY","LT":"LTU","LU":"LUX","MO":"MAC","MK":"MKD","MG":"MDG","MW":"MWI","MY":"MYS","MV":"MDV","ML":"MLI","MT":"MLT","MH":"MHL","MR":"MRT","MU":"MUS","MX":"MEX","FM":"FSM","MD":"MDA","MN":"MNG","ME":"MNE","MA":"MAR","MZ":"MOZ","MM":"MMR","NA":"NAM","NP":"NPL","NL":"NLD","NZ":"NZL","NI":"NIC","NE":"NER","NG":"NGA","NO":"NOR","OM":"OMN","PK":"PAK","PW":"PLW","PA":"PAN","PG":"PNG","PY":"PRY","PE":"PER","PH":"PHL","PL":"POL","PT":"PRT","PR":"PRI","QA":"QAT","RO":"ROU","RU":"RUS","RW":"RWA","WS":"WSM","SM":"SMR","ST":"STP","SA":"SAU","SN":"SEN","RS":"SRB","SC":"SYC","SL":"SLE","SG":"SGP","SK":"SVK","SI":"SVN","SB":"SLB","ZA":"ZAF","SS":"SSD","ES":"ESP","LK":"LKA","KN":"KNA","LC":"LCA","VC":"VCT","SD":"SDN","SR":"SUR","SZ":"SWZ","SE":"SWE","CH":"CHE","SY":"SYR","TW":"TWN","TJ":"TJK","TZ":"TZA","TH":"THA","TL":"TLS","TG":"TGO","TO":"TON","TT":"TTO","TN":"TUN","TR":"TUR","TM":"TKM","TV":"TUV","UG":"UGA","UA":"UKR","AE":"ARE","GB":"GBR","US":"USA","UY":"URY","UZ":"UZB","VU":"VUT","VE":"VEN","VN":"VNM","YE":"YEM","ZM":"ZMB","ZW":"ZWE"}
#... stuff I will avoid
avoid=['unkn.',' unkn.','unkn. ', ' unkn. ','Total']

#We also want to avoid empty flows or write a line with a country would would be both origin and destination
with codecs.open("data.json", "w", "utf-8-sig") as f:
    f.write('[')
    for iso2 in col_list:
        if iso2 not in avoid:
            for iso2_2 in col_list:
                if iso2_2 not in avoid:
                    data=dataset[iso2][iso2_2]
                    if numpy.isnan(data): data="null"
                    if iso2_2!=iso2:
                        if data!="null":
                            if data!=0:
                                f.write('{"origin":"'+str(iso3_country_dict[iso2_iso3_dict[iso2]])+'","destination":"'+str(iso3_country_dict[iso2_iso3_dict[iso2_2]])+'","value":'+str(data)+'},')

    
    f.write('{"origin":"","destination":"","value":null}]')

You can browse (and use) the resulting JSON file on https://api.myjson.com/bins/17gzk7

### Creating the Dataviz
Once we have a nice JSON, we can use different dataviz library. 

#### 1. Using JavaScript Library Highcharts
Since I like it, and have worked  extensively with it, I started with [Highcharts](https://www.highcharts.com/). 
Early this year I worked on a demo for mapping Asylum seekers using data from UNHCR. I wrote a tutorial you can find here: https://www.highcharts.com/blog/post/illustrating-flight-patterns-flow-map-open-data/ 

Here is the same principle applied to Posted Workers in the EU

In [34]:
from IPython.display import HTML
HTML('''<iframe width="100%" height="790" src="https://jsfiddle.net/ThomasRoca/ht80ywmc/embedded/result/" allowfullscreen="allowfullscreen" frameborder="0"></iframe>''')

#### 2. Using Microsoft PowerBI

Since not every one actually loves coding or/and JavaScript, a straightforward way to make sense of complexe a database is  [Microsoft PowerBI](https://powerbi.microsoft.com). It allows creating dashboards with inter-connected data representation which would be difficult to code - a bit like [CrossFilter](http://square.github.io/crossfilter/).

*NB. Note that I work for Microsoft and that alternatives to PowerBI exists. That said I find PowerBi super easy and powerfull*

Using PowerBI, you can plug in the JSON file and transform it into a table to be used in PowerBI. You can then use available representations or even custom ones. Here is the dashboard I built in a few minutes with PowerBI:

In [33]:
from IPython.display import HTML
HTML('''<iframe width="800" height="600" src="https://msit.powerbi.com/view?r=eyJrIjoiZjQ1OTNmMjUtNzEzNi00N2VlLWExOTEtZTc4NDhmYWU4NDkxIiwidCI6IjcyZjk4OGJmLTg2ZjEtNDFhZi05MWFiLTJkN2NkMDExZGI0NyIsImMiOjV9" frameborder="0" allowFullScreen="true"></iframe>
''')